In [26]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from typing import Annotated
import requests

In [24]:
# tool creation

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
    """This function fetches the currency conversion factor between given base currency and a target currency"""
    url = f'https://v6.exchangerate-api.com/v6/4ea038bf196f0c1379803760/pair/{base_currency}/{target_currency}'
    response = requests.get(url)
    return response.json()

In [4]:
get_conversion_factor.invoke({'base_currency':'USD', 'target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1756857601,
 'time_last_update_utc': 'Wed, 03 Sep 2025 00:00:01 +0000',
 'time_next_update_unix': 1756944001,
 'time_next_update_utc': 'Thu, 04 Sep 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 88.1036}

In [29]:
from langchain_core.tools import tool

@tool
def convert(base_currency_value: float, conversion_rate: float) -> float:
    """
    Given a base currency value and a conversion rate, this function calculates
    the target currency value.
    """
    return base_currency_value * conversion_rate


In [30]:
get_conversion_factor.invoke({'base_currency':'USD', 'target_currency': 'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1756857601,
 'time_last_update_utc': 'Wed, 03 Sep 2025 00:00:01 +0000',
 'time_next_update_unix': 1756944001,
 'time_next_update_utc': 'Thu, 04 Sep 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 88.1036}

In [31]:
convert.invoke({'base_currency_value': '10', 'conversion_rate': '88.1036'})


881.0360000000001

In [32]:
# tool binidng
llm = ChatOpenAI()

In [33]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [34]:
messages = [HumanMessage('What is the conversion factor between USD and IND, and based on that can you convert 10 usd to inr')]

In [35]:
messages

[HumanMessage(content='What is the conversion factor between USD and IND, and based on that can you convert 10 usd to inr', additional_kwargs={}, response_metadata={})]

In [36]:
ai_messages = llm_with_tools.invoke(messages)

In [37]:
ai_messages.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'call_2BGyAXCDhNhbyTpHljwNjWoH',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10, 'conversion_rate': 73.68},
  'id': 'call_21g2gVwiDujB6RvTzwakvksd',
  'type': 'tool_call'}]

In [47]:
import json
for tool_call  in ai_messages.tool_calls:
    if tool_call['name'] == 'get_conversion_factor':
        tool_message1 = get_conversion_factor.invoke(tool_call)
        print(tool_message1)

# fetching the conversion rate
conversion_rate = (json.loads(tool_message1.content)['conversion_rate'])

# appending thw tool message to message list
messages.append(tool_message1)
# execute the 2nd tool using the conversion rate from tool 1 
if tool_call['name'] == 'convert':
    # fetching the curent arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_messages2 = convert.invoke(tool_call)
    messages.append(tool_messages2)

content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": "https://www.exchangerate-api.com/terms", "time_last_update_unix": 1756857601, "time_last_update_utc": "Wed, 03 Sep 2025 00:00:01 +0000", "time_next_update_unix": 1756944001, "time_next_update_utc": "Thu, 04 Sep 2025 00:00:01 +0000", "base_code": "USD", "target_code": "INR", "conversion_rate": 88.1036}' name='get_conversion_factor' tool_call_id='call_2BGyAXCDhNhbyTpHljwNjWoH'


In [50]:
messages

[HumanMessage(content='What is the conversion factor between USD and IND, and based on that can you convert 10 usd to inr', additional_kwargs={}, response_metadata={}),
 ToolMessage(content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": "https://www.exchangerate-api.com/terms", "time_last_update_unix": 1756857601, "time_last_update_utc": "Wed, 03 Sep 2025 00:00:01 +0000", "time_next_update_unix": 1756944001, "time_next_update_utc": "Thu, 04 Sep 2025 00:00:01 +0000", "base_code": "USD", "target_code": "INR", "conversion_rate": 88.1036}', name='get_conversion_factor', tool_call_id='call_2BGyAXCDhNhbyTpHljwNjWoH'),
 ToolMessage(content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": "https://www.exchangerate-api.com/terms", "time_last_update_unix": 1756857601, "time_last_update_utc": "Wed, 03 Sep 2025 00:00:01 +0000", "time_next_update_unix": 1756944001, "time_next_update_utc": "Thu, 04 Sep 

In [ ]:
llm_with_tools.invoke(messages).content